# Generate intraday bars using dedicated data service - Python

### Overview
Bars are widely used to illustrate price movements for an instrument over a look back period. Commonly used in financial analysis and trading strategies as per a decicion making technical indicator.<br>

This sample demonstrates how to directly request and plot *`intraday bars`*.<br>

It enables to retrieve **on-demand** intraday bars data by calling a dedicated service. 

### Inputs/outputs
Intraday bars sample requires instrument's identifier as per input aswell as a sampling. It returns open-high-low-close (OHLC), volume, vwap.
This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve bars from the dedicated hosted service. The queried endpoint in this script are:
* *IntradayBarsService*: to directly retrieve intraday bars objects from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.type.shared.v1*
    * *systemathics.apis.services.daily.v1*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *mplfinance* as per display package

***

# Run intraday bars sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas mplfinance plotly

In [ ]:
pip install systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.sampling_pb2 as sampling
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.intraday.v1.intraday_bars_pb2 as intraday_bars
import systemathics.apis.services.intraday.v1.intraday_bars_pb2_grpc as intraday_bars_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *intraday bars* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Sampling
* Service called: *IntradayBarsService* in order to retrieve intraday bars data

In [ ]:
# set the instrument
ticker = 'AAPL'
exchange = 'BATS'

# Select sampling between:
# ONE_MINUTE, FIVE_MINUTE,TEN_MINUTE, FIFTEEN_MINUTE, THIRTY_MINUTE, ONE_HOUR
my_sampling = sampling.SAMPLING_FIFTEEN_MINUTE

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
# create the intraday bars request
request = intraday_bars.IntradayBarsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    sampling = my_sampling
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the intraday prices service
        service = intraday_bars_service.IntradayBarsServiceStub(channel)
        
        # process the request
        response = service.IntradayBars(
            request = request, 
            metadata = [('authorization', token)]
        )
        
    print("Total bars retrieved: ",len(response.data))
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

### Step 4: Visualize data

#### 4.1 Retrieve intraday bars data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
#pPrepare the data frame content
dates=[datetime.fromtimestamp(b.time_stamp.seconds) for b in response.data]
opens = [b.open for b in response.data]
highs = [b.high for b in response.data]
lows = [b.low for b in response.data]
closes = [b.close for b in response.data]
volumes = [b.volume for b in response.data]
counts = [b.count for b in response.data]
vwaps = [b.vwap for b in response.data]
scores = [b.score for b in response.data]

d = {'Date': dates, 'Open': opens, 'High': highs, 'Low' : lows,'Close': closes, 'Volume': volumes, 'Count': counts , 'Vwap': vwaps , 'Score': scores }

In [ ]:
# create pandas dataframe
df = pd.DataFrame(data=d)

In [ ]:
# visualize dataframe
df

#### 4.2 Export intraday bars as csv

In [ ]:
os.makedirs('Export', exist_ok=True)
df.to_csv('Export/{0}-{1}.csv'.format(ticker,exchange), index=False)

#### 4.3 Plot intraday bars with mplfinance
The following code snippets display daily bars' request results with the package of your choice.<br>*`mplfinance`* is used as per open source display package.

In [ ]:
# set number of days
N = 100

In [ ]:
# set date as dataframe index
df = df.set_index('Date')

In [ ]:
import mplfinance as mpf

mpf.plot(df.tail(N), figratio=(40,15),
         type = 'candle', 
         title=('Daily bars for {0}-{1}: last {2} days'.format(ticker,exchange,N)),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

#### 4.4 Plot daily bars with plotly
The following code snippets display daily bars' request results with the package of your choice.<br>*`plotly`* is used as per open source display package.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name="Bars"), secondary_y=True)
fig.add_trace(go.Bar(x=df.index, y=df['Volume'], name="Volume"), secondary_y=False)

fig.update_layout(title = 'Daily bars for {0}-{1}'.format(ticker,exchange))
fig.update_xaxes(title_text = 'Date')
fig.layout.yaxis2.showgrid=False
fig.show()